In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [2]:
# n = len(documents)

# for i in range(n):
#     documents[i]["id"]=i

In [3]:
import hashlib

In [4]:
def generate_document_id(doc):
    combined = f"{doc["course"]}-{doc["question"]}-{doc["text"][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [5]:
for doc in documents:
    doc["id"] = generate_document_id(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [7]:
from collections import defaultdict

In [8]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc["id"]
    hashes[doc_id].append(doc)

In [9]:
len(hashes), len(documents)

(947, 948)

In [10]:
for k, values in hashes.items():
    if len(values) >1:
        print(k, len(values))

593f7569 2


In [11]:
hashes["593f7569"]

[{'text': "They both do the same, it's just less typing from the script.\nAsked by Andrew Katoch, Added by Edidiong Esu",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'},
 {'text': "They both do the same, it's just less typing from the script.",
  'section': '6. Decision Trees and Ensemble Learning',
  'question': 'Does it matter if we let the Python file create the server or if we run gunicorn directly?',
  'course': 'machine-learning-zoomcamp',
  'id': '593f7569'}]

In [12]:
import json

In [13]:
with open("document-with-ids.json", "wt") as f_out:
    json.dump(documents, f_out, indent=2)

In [14]:
!head document-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [15]:
prompt_template = """
    You emulate a student taking our course. 
    Generate 5 users question based on the provided FAQ record. The record should conatain the answer 
    to the questions, and the questions should be completed and not too short.
    If possible, use as fewer words as possible from the record.
    
    
    The record:
    
    section: {section}
    question: {question}
    answer: {text}
    
    Provide the output in parsable JSON without using code blocks:
    
    ["question1", "question2" , ..., "question5"] 

""".strip()

In [16]:
from openai import OpenAI
import os

In [17]:
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
key = os.getenv("OPENAI_API_KEY")

In [19]:
client = OpenAI(api_key=key)

In [20]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    json_response = response.choices[0].message.content
    
    return json_response


In [21]:
doc = documents[2]
generate_questions(doc)

'["Is it possible to enroll in the course after it has begun?", "If I miss the registration, can I still submit assignments?", "Are there any deadlines I should be concerned about for final projects?", "Can I join the course later and still complete my homework?", "What happens if I wait until the last minute to submit my final project?"]'

In [22]:
from tqdm.auto import tqdm

In [23]:
results = {}

In [24]:
for doc in tqdm(documents):
    doc_id = doc["id"]
    if doc_id in results:
        continue
    questions = generate_questions(doc)
    results[doc_id]=questions

  0%|          | 0/948 [00:00<?, ?it/s]

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-ZkgZdL7ZvyzTJZRYjSpd9tDv on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}